# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch
import esm
import gc
import os
import pickle
from tqdm import tqdm
from copy import deepcopy

import numpy as np
from numpy.random import default_rng
from torch.distributions import relaxed_bernoulli
import torch.nn.functional as F

# PLOTTING
import matplotlib.pyplot as plt
from matplotlib.colors import CenteredNorm
import matplotlib.cm as cm

In [ ]:
#| export
from DiffPALM.gumbel_sinkhorn_utils import *
from DiffPALM.msa_transformer_utils import read_msa, read_sequence

if torch.cuda.is_available():
    if torch.cuda.device_count() == 1:
        DEVICE = "cuda"
    else:
        device_no = input("Choose CUDA device number: ")
        if device_no == "":
            DEVICE = "cpu"
        else:
            DEVICE = f"cuda:{device_no}"
# Uncomment when all needed ops are available for MPS
# elif torch.backends.mps.is_built():
#     DEVICE = "mps"
else:
    DEVICE = "cpu"
    
DEVICE = torch.device(DEVICE)
print(f"DEVICE = {DEVICE}")

msa_transformer, msa_alphabet = esm.pretrained.esm_msa1b_t12_100M_UR50S()
msa_transformer = msa_transformer.eval().to(DEVICE)
msa_batch_converter = msa_alphabet.get_batch_converter()

In [ ]:
#| export

def DCN(x):
    return x.detach().clone().cpu().numpy()


class PermutationsMixin:
    mask_idx = 32

    def _init_log_alpha(self, skip=False):
        if not skip:
            if self._is_depth_ndarray:
                # Permutations restricted to species
                self.log_alpha = [
                    (self.std_init * torch.randn(d, d, device=self.device)).requires_grad_(True)
                    for d in self._effective_depth_not_fixed
                ]
            else:
                # Permutations over all input sequences
                d = self._effective_depth_not_fixed
                self.log_alpha = (self.std_init * torch.randn(d, d, device=self.device)).requires_grad_(True)

    def _validator(self, input_left, input_right, fixed_pairings=None):
        self._is_depth_ndarray = type(self.depth) == np.ndarray
        # Validate input MSAs
        depth_left, length_left, alphabet_size_left = input_left.shape[1:]
        depth_right, length_right, alphabet_size_right = input_right.shape[1:]
        length_left -= 1
        length_right -= 1
        if depth_left != depth_right:
            raise ValueError(f"Depth mismatch between left MSA ({depth_left}) and right MSA "
                             f"({depth_right})")
        if alphabet_size_left != alphabet_size_right:
            raise ValueError("Input MSAs must have the same alphabet size/")
        self._alphabet_size = alphabet_size_left

        # Define oh vector for mask token
        self._vec_mask = torch.zeros(self._alphabet_size, device=self.device)
        self._vec_mask[self.mask_idx] = 1

        # Validate depth attribute
        self._depth_total = self.depth if not self._is_depth_ndarray else sum(self.depth)
        if depth_left != self._depth_total:
            raise ValueError(f"Input MSAs have depth {depth_left} but model expects a total "
                             f"depth of {self._depth_total}")
        self._length_left, self._length_right = length_left, length_right
        self._length = length_left + length_right

        self._depth = self.depth if self._is_depth_ndarray else [self.depth]
        self._effective_mask_not_fixed = torch.ones(self._depth_total, self._depth_total,
                                                    dtype=torch.bool, device=self.device)

        # Create masking array for non-fixed partial rows in concatenated MSA
        self._effective_mask_not_fixed_cat = torch.ones(
            1, self._depth_total, self._length, dtype=torch.bool, device=self.device
        )
        if fixed_pairings is not None:
            if self._is_depth_ndarray:
                if len(fixed_pairings) != len(self.depth):
                    raise ValueError(f"`fixed_pairings` has length {len(fixed_pairings)} but "
                                     f"there are {self.depth} species.")
                _fixed_pairings = fixed_pairings
            else:
                _fixed_pairings = [fixed_pairings]

            start = 0
            self._effective_depth_not_fixed = []
            self._effective_fixed_pairings_zip = []
            for (species_idx,
                 (species_size,
                  species_fixed_pairings)) in enumerate(zip(self._depth, _fixed_pairings)):
                # Check uniqueness of pairs (i, j)
                n_fixed = len(set(species_fixed_pairings))
                if len(species_fixed_pairings) > n_fixed:
                    raise ValueError("Repeated indices for fixed pairings at species "
                                     f"{species_idx}: {species_fixed_pairings}")
                fixed_pairings_arr = np.zeros((species_size, species_size), dtype=int)
                if species_fixed_pairings:
                    species_fixed_pairings_zip = tuple(zip(*species_fixed_pairings))
                else:
                    # species_fixed_pairings is an empty list
                    species_fixed_pairings_zip = (tuple(), tuple())
                try:
                    fixed_pairings_arr[species_fixed_pairings_zip] = 1
                except IndexError:
                    raise ValueError(
                        f"Fixed pairings indices out of bounds: passed {species_fixed_pairings} "
                        f"for species {species_idx} with size {species_size}."
                    )
                partial_sum_0 = fixed_pairings_arr.sum(axis=0)
                partial_sum_1 = fixed_pairings_arr.sum(axis=1)
                if (partial_sum_0 > 1).any() or (partial_sum_1 > 1).any():
                    raise ValueError(
                        f"Passed fixed pairings for species {species_idx} are either not one-one "
                        "or a multiply-defined mapping from row to column indices: "
                        f"{species_fixed_pairings}"
                    )
                for i, j in species_fixed_pairings:
                    self._effective_mask_not_fixed[start + i, :] = False
                    self._effective_mask_not_fixed[:, start + j] = False
                total_minus_fixed = species_size - n_fixed
                # If species_size - n_fixed <= 1 then actually everything is fixed
                self._effective_depth_not_fixed.append(
                    int(total_minus_fixed > 1) * total_minus_fixed
                )
                if total_minus_fixed == 1:
                    # Deduce implicitly fixed pair
                    i_implicit, j_implicit = np.argmin(partial_sum_1), np.argmin(partial_sum_0)
                    self._effective_mask_not_fixed[start + i_implicit, :] = False
                    self._effective_mask_not_fixed[:, start + j_implicit] = False
                    species_fixed_pairings_zip = (species_fixed_pairings_zip[0] + (i_implicit,),
                                                  species_fixed_pairings_zip[1] + (j_implicit,))
                self._effective_fixed_pairings_zip.append(species_fixed_pairings_zip)
                start += species_size
            start = 0
            for (species_size,
                 (rows_fixed, cols_fixed)) in zip(self._depth, self._effective_fixed_pairings_zip):
                self._effective_mask_not_fixed_cat[:, start:, ...][:, rows_fixed, :length_left] = \
                    False
                self._effective_mask_not_fixed_cat[:, start:, ...][:, cols_fixed, length_left:] = \
                    False
                start += species_size
        else:
            self._effective_depth_not_fixed = self._depth
            self._effective_fixed_pairings_zip = None

        self._default_target_idx = torch.arange(
            self._depth_total, dtype=torch.int64, device=self.device
        )

    def plot_real_time(self,
                       it,
                       gs_matching_mat_np,
                       gs_mat_np,
                       list_idx,
                       target_idx,
                       list_log_alpha,
                       losses,
                       batch_size,
                       epochs,
                       lr,
                       tar_loss,
                       new_noise_factor,
                       folder_path,
                       showfig,
                       only_loss_plot):
        n_correct = [sum(idx == target_idx) for idx in list_idx[::batch_size]]

        cmap = cm.get_cmap("Blues")
        normalizer = None
        fig, axes = plt.subplots(figsize=(30, 5),
                                 ncols=5,
                                 constrained_layout=True)

        null_model = 1
        if self._is_depth_ndarray:
            null_model = len(self.depth)
            _depth = [0] + list(np.cumsum(self.depth))
            for k in range(1, len(_depth)):
                for ii in range(2):
                    elem, elem1 = _depth[k - 1], _depth[k]
                    axes[ii].plot([elem - .5, elem1 - .5, elem1 - .5, elem - .5],
                                  [elem - .5, elem - .5, elem1 - .5, elem1 - .5],
                                  color="r")
                    axes[ii].plot([elem - .5, elem - .5, elem1 - .5, elem1 - .5],
                                  [elem - .5, elem1 - .5, elem1 - .5, elem - .5],
                                  color="r")

        ims_soft = axes[0].imshow(gs_mat_np, cmap=cmap, norm=normalizer)
        axes[0].set_title(f"Soft {it // batch_size}")
        axes[1].imshow(gs_matching_mat_np, cmap=cmap, norm=normalizer)
        axes[1].set_title("Hard")

        curr_log_alpha = list_log_alpha[-1]
        ims_log_alpha = axes[2].imshow(curr_log_alpha, norm=CenteredNorm(), cmap=cm.bwr)
        axes[2].set_title("Log-alpha")

        prev_log_alpha = list_log_alpha[-2] if len(list_log_alpha) > 1 else curr_log_alpha
        diff_log_alpha = curr_log_alpha - prev_log_alpha
        if np.nansum(np.abs(diff_log_alpha)):
            ims_log_alpha_diff = axes[3].imshow(diff_log_alpha, norm=CenteredNorm(), cmap=cm.bwr)
            cb3 = fig.colorbar(ims_log_alpha_diff, ax=axes[3], shrink=0.8)
        else:
            ims_log_alpha_diff = axes[3].imshow(np.zeros_like(diff_log_alpha), cmap=cm.bwr)
            cb3 = None
        axes[3].set_title("Log-alpha diff")

        avg_loss = np.mean(np.array(losses).reshape(-1, batch_size), axis=1)
        axes[4].plot(avg_loss, color="b", linewidth=1)
        ax3_2 = None
        if not only_loss_plot:
            if tar_loss is not None:
                axes[4].axhline(tar_loss, color="b", linewidth=2)
            diff = avg_loss[0] - tar_loss
            axes[4].set_ylim([tar_loss - 0.6 * diff, avg_loss[0] + 0.5 * diff])
            ax3_2 = axes[4].twinx()
            ax3_2.set_ylabel("No. of correct pairs", color="red")
            ax3_2.plot(n_correct, color="red", linewidth=1)
            ax3_2.axhline(null_model, color="red", linewidth=2)
            ax3_2.tick_params(axis="y", labelcolor="red")
        axes[4].set_ylabel("Loss")
        axes[4].set_xlim([0, epochs])
        axes[4].set_title(f"lr: {lr:.3g}, noise:{new_noise_factor:.3g}")
        cb1 = fig.colorbar(ims_soft, ax=axes[0], shrink=0.8)
        cb2 = fig.colorbar(ims_log_alpha, ax=axes[2], shrink=0.8)
        if folder_path is not None:
            if not folder_path.exists():
                os.mkdir(folder_path)
            fig.savefig(folder_path / f"epoch={it // batch_size}.svg")
        if showfig:
            plt.show()
        # Clear the current axes.
        plt.cla()
        # Clear the current figure.
        plt.clf()
        # Closes all the figure windows.
        plt.close('all')
        plt.close(fig)
        del fig, cb1, cb2, cb3, ax3_2, ims_soft, ims_log_alpha, ims_log_alpha_diff
        gc.collect()

In [ ]:
#| export

class Permutations_MLM(torch.nn.Module, PermutationsMixin):
    """
    Class which permutes the pairs between two concatenated MSAs and backpropagate the loss on
    the amino-acids directly to the permutation matrix to get the correct permutation of interacting
    pairs. The input to the forward pass must be MSAs of one-hot encodings of shape (1,D,L,alphabet)
    -- i.e., every amino-acid in the MSA is a one-hot vector.
    """

    def __init__(self, device, depth, *, p_mask=0.7, only_mask_left=True, seed=0, **kwargs):
        super().__init__()
        self.device = device
        # Depth of MSAs
        self.depth = depth
        # Masking probability of tokens
        self.p_mask = p_mask
        self.only_mask_left = only_mask_left
        # Loss function
        self.Loss = torch.nn.CrossEntropyLoss()
        self.Loss_flat = torch.nn.CrossEntropyLoss(reduction="none")
        # Set random seed
        self.seed = seed
        torch.manual_seed(seed)

    def _validate_p_mask(self):
        if self.only_mask_left or not isinstance(self.p_mask, list):
            self._p_mask = [self.p_mask]  # Expected to be a single float or single tensor
        else:  # Expect case of list of tensor
            self._p_mask = self.p_mask

    def forward(self, input_ord, input_right, positive_examples=None, p_mask=0.7,
                offset_pos_emb_from=None, offset_pos_emb_by=0, only_masked_part=True):
        """
        Compute the output logits of randomly masked tokens.

        input_left: variable input (MSA to mask and permute)    --> (B, D, L1 + 1, 33)
        input_right: fixed input (MSA of pairs: no masking)      --> (B, D, L2 + 1, 33)
        positive_examples: if not None it's a concatenation of correct pairs to use as context
                           (not masked)                        --> (B, D, L1 + L2 + 1, 33)
        """
        # print(ii, torch.cuda.memory_allocated()/10**9, torch.cuda.memory_reserved()/10**9)

        # One-hot vector to use as mask in the MSA
        vec_mask = torch.zeros(input_ord.shape[-1]).to(self.device)
        vec_mask[self.mask_idx] = 1
        # Create masking array # --> (B,D,L1)

        # Mask input MSA in the positions specified by `mask` matrix (using`vec_mask`)
        if self.only_mask_left:
            input_mask = input_ord.clone()
            mask = (torch.rand(input_ord[..., 1:, 0].shape) < p_mask).to(self.device)
        else:
            input_mask = torch.cat((input_ord, input_right[..., 1:, :]), dim=2).clone()
            mask = (torch.rand(input_mask[..., 1:, 0].shape) < p_mask).to(self.device)

        if self._effective_fixed_pairings_zip is not None:
            start = 0
            for (species_size,
                 (_, cols_fixed)) in zip(self._depth, self._effective_fixed_pairings_zip):
                mask[:, start:, ...][:, cols_fixed, ...] = False
                start += species_size

        input_mask[..., 1:, :][mask] = vec_mask
        if self.only_mask_left:
            # Concatenate masked MSA with fixed MSA (not masked)
            input_mask = torch.cat((input_mask, input_right[..., 1:, :]), dim=2)
        # Add positive examples (correct pairs) on top of input MSA
        if positive_examples is not None:
            input_mask = torch.cat((positive_examples, input_mask), dim=1)

        # Compute output logits
        results = msa_transformer(input_mask, repr_layers=[12],
                                  offset_pos_emb_from=offset_pos_emb_from,
                                  offset_pos_emb_by=offset_pos_emb_by)
        logits = results["logits"]
        if only_masked_part and self.only_mask_left:
            # Restrict to logits of masked MSA
            logits = logits[:, :, 1:input_ord.shape[2], :]
        else:
            # Output logits of both MSAs
            logits = logits[:, :, 1:, :]
        if positive_examples is not None:
            logits = logits[:, positive_examples.shape[1]:, :, :]

        return logits, mask

    # @profile
    def train(self,
              input_left,
              input_right,
              fixed_pairings=None,  # [[(i, j), ...], ...]
              positive_examples=None,
              std_init=0.1,
              epochs=1,
              optimizer_name="Adam",
              optimizer_kwargs=None,
              adaptive_weight_decay=False,
              ord_reg=None,
              lambda_reg=1e-3,
              tau=1.,
              n_sink_iter=20,
              noise=True,
              noise_std=False,
              noise_factor=1.,
              noise_scheduler=False,
              scheduler_name="ReduceLROnPlateau",
              scheduler_kwargs=None,
              batch_size=1,
              use_rand_perm=False,
              keep_working_hard_perm=False,
              mean_centering=False,
              hard_loss=False,
              offset_pos_emb_from=None,
              offset_pos_emb_by=0,
              tar_loss=None,
              folder_path=None,
              time_processes=False,
              save_checkpoint=False,
              early_stopping=False,
              showfig=False,
              init_log_alpha=True,
              only_loss_plot=False,
              **kwargs):
        self._validator(input_left, input_right, fixed_pairings=fixed_pairings)
        if not sum(self._effective_depth_not_fixed):
            print("No parameters available to optimize, pairings are fixed by the input.")
            return None
        self.std_init = std_init

        self._validate_p_mask()

        base_params = {"noise": noise, "noise_std": noise_std}
        sinkhorn_params = {"tau": tau, "n_iter": n_sink_iter}

        # Initialize log_alpha given fixed pairings
        if init_log_alpha:
            self._init_log_alpha()

        if save_checkpoint and (folder_path is None):
            raise ValueError("Cannot save checkpoint if `folder_path` is None.")

        # ------------------------------------------------------------------------------------------
        ## Useful functions
        # ------------------------------------------------------------------------------------------
        def _identity(x, **kwargs):
            return x

        def _impl_func(func):
            # Apply `func` either to the whole log_alpha or just to the blocks for permutation over
            # species
            def _case_restricted_to_species(log_alpha, **params):
                # Block matrix for permutations within species
                noise_mat = params.pop("noise_mat")  # List of noise matrices
                rand_perm = params.pop("rand_perm")  # List of random permutations
                return torch.block_diag(
                    *[func(la, noise_mat=nm, rand_perm=rp, **params) if la.size(0) else la
                      for la, nm, rp in zip(log_alpha, noise_mat, rand_perm)]
                )

            def _case_unrestricted(log_alpha, **params):
                # Else, permutation matrix over all sequences
                # TODO INCLUDE NOISE AND RANDOM PERMUTATIONS
                return func(log_alpha, **params) if log_alpha.size(0) else log_alpha

            if self._is_depth_ndarray:
                return _case_restricted_to_species
            return _case_unrestricted

        def _keep_working_hard_perm_func(gs_matching_mat, prev_gs_matching_mat=None):
            # Keep same permutation if the frobenius norm of the previous one is bigger or
            # equal than the new one (without Noise)
            if prev_gs_matching_mat is None:
                pass
            elif self._is_depth_ndarray:
                d_start = 0
                for d, la in zip(self.depth, self.log_alpha):
                    d_end = d_start + d
                    gs_matching_submat = gs_matching_mat[d_start:d_end, d_start:d_end]
                    prev_gs_matching_submat = prev_gs_matching_mat[d_start:d_end, d_start:d_end]
                    frob = gs_matching_submat.T.matmul(la).diagonal().sum()
                    prev_frob = prev_gs_matching_submat.T.matmul(la).diagonal().sum()
                    if prev_frob >= frob:
                        # Frobenius norm should be maximised, use old matching if not worse
                        gs_matching_mat[d_start:d_end, d_start:d_end] = prev_gs_matching_submat
                    d_start = d_end
            else:
                frob = gs_matching_mat.T.matmul(self.log_alpha).diagonal().sum()
                prev_frob = prev_gs_matching_mat.T.matmul(self.log_alpha).diagonal().sum()
                if prev_frob >= frob:
                    # Frobenius norm should be maximised, use old matching if not worse
                    gs_matching_mat[:, :] = prev_gs_matching_mat

            return gs_matching_mat

        def _noise_mat():
            if self._is_depth_ndarray:
                if noise:
                    return [sample_uniform(la.size()).to(self.device) for la in self.log_alpha]
                return [None for la in self.log_alpha]
            elif noise:
                return sample_uniform(self.log_alpha.size()).to(self.device)
            return None

        def _rand_perm():
            if self._is_depth_ndarray:
                if use_rand_perm:
                    rand_perm = []
                    for la in self.log_alpha:
                        n = la.shape[0]
                        rp = []
                        for _ in range(2):
                            rp_i = torch.zeros_like(la, device=self.device)
                            rp_i[torch.arange(n), torch.randperm(n)] = 1
                            rp.append(rp_i)
                        rand_perm.append(rp)
                else:
                    rand_perm = [None] * len(self.log_alpha)
            else:
                if use_rand_perm:
                    rand_perm = []
                    n = self.log_alpha.shape[0]
                    for _ in range(2):
                        rand_perm_i = torch.zeros_like(self.log_alpha, device=self.device)
                        rand_perm_i[torch.arange(n), torch.randperm(n)] = 1
                        rand_perm.append(rand_perm_i)
                else:
                    rand_perm = None
            return rand_perm

        if keep_working_hard_perm and not noise:
            processing_ = _keep_working_hard_perm_func
        else:
            processing_ = _identity

        gumbel_matching_ = _impl_func(gumbel_matching)
        gumbel_sinkhorn_ = _impl_func(gumbel_sinkhorn)

        # ------------------------------------------------------------------------------------------
        ## Input MSAs and initial variables
        # ------------------------------------------------------------------------------------------
        input_left = input_left.to(self.device).requires_grad_(True)
        input_right = input_right.to(self.device).requires_grad_(True)

        if time_processes:
            delta1, delta2, delta3 = [], [], []
            start = torch.cuda.Event(enable_timing=True)
            end = torch.cuda.Event(enable_timing=True)

        # Lists of parameters
        losses = []
        mats, mats_gs = [], []
        list_idx = []
        list_log_alpha = []
        list_scheduler = []
        gs_matching_mat = None
        target_idx = torch.arange(self._depth_total, dtype=torch.float, device=self.device)
        target_idx_np = DCN(target_idx)
        # ------------------------------------------------------------------------------------------
        ## Initializations
        # ------------------------------------------------------------------------------------------
        # Optimizer
        if self._is_depth_ndarray:
            optimizer_params = [{"params": la} for la in self.log_alpha]
        else:
            optimizer_params = [{"params": self.log_alpha}]
        optimizer_kwargs_ = {} if optimizer_kwargs is None else deepcopy(optimizer_kwargs)
        if adaptive_weight_decay:
            # Divide the weight decay parameter with the square of the number of sequences
            # in each block over ten.
            wd = optimizer_kwargs_.pop("weight_decay")
            for elem in optimizer_params:
                elem["weight_decay"] = wd / (max(elem["params"].shape[0], 1) / 10)**2
        optimizer_cls = getattr(torch.optim, optimizer_name, torch.optim.SGD)
        optimizer = optimizer_cls(optimizer_params, **optimizer_kwargs_)
        # Scheduler
        if scheduler_name is not None:
            if type(scheduler_name) != list:
                scheduler_name = [scheduler_name]
                scheduler_kwargs = [scheduler_kwargs]
            scheduler_ = {}
            for elem, kwarg in zip(scheduler_name, scheduler_kwargs):
                scheduler_cls = getattr(
                    torch.optim.lr_scheduler, elem, torch.optim.lr_scheduler.ReduceLROnPlateau
                )
                scheduler_[elem] = scheduler_cls(optimizer, **kwarg)
        else:
            scheduler_cls = None

        # ------------------------------------------------------------------------------------------
        ## Start training
        # ------------------------------------------------------------------------------------------
        iterations = epochs * batch_size
        with torch.set_grad_enabled(True):
            optimizer.zero_grad()
            for i in tqdm(range(iterations + batch_size)):
                # ----------------------------------------------------------------------------------
                ## Noise Matrices for permutations
                # ----------------------------------------------------------------------------------
                if i % batch_size == 0:
                    # Save log_alpha
                    _log_alpha = torch.full((self._depth_total, self._depth_total), torch.nan,
                                            dtype=torch.float, device=self.device)
                    _log_alpha.masked_scatter_(
                        self._effective_mask_not_fixed,
                        torch.block_diag(*self.log_alpha)
                        if self._is_depth_ndarray else self.log_alpha
                    )
                    list_log_alpha.append(DCN(_log_alpha))
                    # Create new noise matrices and random shufflings only every `batch_size` iterations
                    noise_mat = _noise_mat()
                    rand_perm = _rand_perm()
                # Set value of noise correction
                new_noise_factor = 0
                if noise:
                    new_noise_factor = noise_factor
                    if noise_scheduler:
                        new_noise_factor = \
                            noise_factor * optimizer.param_groups[0]["lr"] / optimizer_kwargs["lr"]

                # Timing
                if time_processes:
                    start.record()

                # Mean-center log-alphas
                if mean_centering:
                    with torch.no_grad():
                        for la in self.log_alpha:
                            la[...] -= la.mean()

                # ----------------------------------------------------------------------------------
                ## Compute permutation matrices
                # ----------------------------------------------------------------------------------
                params = {**base_params,
                          **{"noise_mat": noise_mat, "noise_factor": new_noise_factor,
                             "rand_perm": rand_perm}}
                gs_matching_mat_not_fixed = processing_(gumbel_matching_(self.log_alpha, **params),
                                                        prev_gs_matching_mat=gs_matching_mat)
                params.update(sinkhorn_params)
                gs_mat_not_fixed = gumbel_sinkhorn_(self.log_alpha, **params)
                if fixed_pairings is not None:
                    gs_matching_mat = torch.zeros(self._depth_total, self._depth_total,
                                                  dtype=torch.float, device=self.device)
                    _gs_mat = torch.zeros_like(gs_matching_mat, device=self.device, requires_grad=True)
                    gs_mat = _gs_mat.clone()
                    start = 0
                    for (species_size,
                         species_fixed_pairings) in zip(self._depth,
                                                        self._effective_fixed_pairings_zip):
                        gs_matching_mat[start:, start:][species_fixed_pairings] = 1.
                        gs_mat[start:, start:][species_fixed_pairings] = 1.
                        start += species_size
                    gs_mat.masked_scatter_(self._effective_mask_not_fixed, gs_mat_not_fixed)
                    gs_matching_mat.masked_scatter_(self._effective_mask_not_fixed,
                                                    gs_matching_mat_not_fixed)
                else:
                    gs_matching_mat = gs_matching_mat_not_fixed
                    gs_mat = gs_mat_not_fixed
                # Save hard or soft permutation matrix
                if i % batch_size == 0:
                    mats.append(DCN(gs_matching_mat if hard_loss else gs_mat))
                    mats_gs.append(DCN(gs_mat))
                # Save permuted indexes
                list_idx.append(DCN(torch.einsum("pq,p->q", (gs_matching_mat, target_idx))))
                # ----------------------------------------------------------------------------------
                ## Permute sequences of input_left using detach trick to backprop only on soft perm.
                # ----------------------------------------------------------------------------------
                input_left_ord = MSA_inverse_permutation(input_left, gs_mat)
                if hard_loss or i == iterations:
                    # Compute hard permutation (i == iterations to compute hard permutation also at
                    # end of soft permutations)
                    input_left_ord_hard = MSA_inverse_permutation(input_left, gs_matching_mat)
                    if hard_loss:
                        input_left_ord = \
                            (input_left_ord_hard - input_left_ord).detach() + input_left_ord
                # Timing
                if time_processes:
                    end.record()
                    torch.cuda.synchronize()
                    delta1.append(start.elapsed_time(end))
                    start.record()
                # ----------------------------------------------------------------------------------
                ## Get output logits of MSA Transformer for the permuted, masked input
                # ----------------------------------------------------------------------------------
                loss = torch.tensor(0., device=self.device, requires_grad=True)
                input_for_CE = input_left_ord if self.only_mask_left else torch.cat((input_left_ord, input_right[..., 1:, :]), dim=2)
                for p_mask in self._p_mask:
                    logits, mask = self(
                        input_left_ord, input_right, positive_examples=positive_examples,
                        p_mask=p_mask, offset_pos_emb_from=offset_pos_emb_from,
                        offset_pos_emb_by=offset_pos_emb_by
                    )
                    loss = loss + self.Loss(logits[mask], input_for_CE[..., 1:, :][mask])
                loss = loss / batch_size
                # Timing
                if time_processes:
                    end.record()
                    torch.cuda.synchronize()
                    delta2.append(start.elapsed_time(end))
                    start.record()
                # ----------------------------------------------------------------------------------
                ## Compute pseudoLikelihood loss only on masked positions and save gradients
                # ----------------------------------------------------------------------------------
                pure_loss = loss.item()
                # Add L_p regularization to the loss
                if ord_reg is not None:
                    loss_reg = torch.tensor(0., device=self.device, requires_grad=True)
                    for ord in [ord_reg]:  # add , 2. if you want to implement L2 norm instead of weight decay
                        if self._is_depth_ndarray:          # but then no weight_decay in optimizer
                            for la in self.log_alpha:
                                loss_reg = loss_reg + (la.exp() / tau).pow(ord).mean()
                        else:
                            loss_reg = loss_reg + (self.log_alpha.exp() / tau).pow(ord).mean()
                    loss = loss + lambda_reg * loss_reg
                # L1 - L2
                # loss_reg = torch.tensor(0., device=self.device, requires_grad=True)
                # for jjj in range(len(self.log_alpha)):
                #     loss_reg = loss_reg + (torch.norm(self.log_alpha[jjj], p=1, dim=1) - torch.norm(self.log_alpha[jjj], p=2, dim=1)).sum() + (torch.norm(self.log_alpha[jjj], p=1, dim=0) - torch.norm(self.log_alpha[jjj], p=2, dim=0)).sum()
                # loss = loss + lambda_reg * loss_reg
                # Compute gradients
                loss.backward()
                # Save loss values
                losses.append(pure_loss * batch_size)
                # Timing
                if time_processes:
                    end.record()
                    torch.cuda.synchronize()
                    delta3.append(start.elapsed_time(end))

                #      plot and save at every batch_size     or       no plots and save at last iteration
                if (((i + 1) % batch_size == 0) and showfig) or ((i == iterations + batch_size - 1) and not showfig):
                    self.plot_real_time(
                        i, DCN(gs_matching_mat), DCN(gs_mat), list_idx, target_idx_np,
                        list_log_alpha, losses, batch_size, epochs,
                        optimizer.param_groups[0]["lr"], tar_loss, new_noise_factor, folder_path,
                        showfig, only_loss_plot
                    )
                # ----------------------------------------------------------------------------------
                ## Optimizer and Scheduler step (with gradient accumulation in batches)
                # ----------------------------------------------------------------------------------
                # Compute this every time with exception of last iteration
                if i < iterations:
                    # Gradient Accumulation
                    if ((i + 1) % batch_size == 0) or ((i + 1) == iterations):
                        optimizer.step()
                        optimizer.zero_grad()
                        # Update scheduler
                        if scheduler_name is not None:
                            if len(scheduler_name) == 1:
                                for elem in scheduler_name:
                                    if elem == "ReduceLROnPlateau":
                                        scheduler_[elem].step(sum(losses[-batch_size:]))
                                    else:
                                        scheduler_[elem].step()
                            elif len(scheduler_name) == 2:
                                if (i + 1) / iterations < scheduler_kwargs[0]["pct_start"]:
                                    scheduler_[scheduler_name[0]].step()
                                elif scheduler_name[1] == "ReduceLROnPlateau":
                                    scheduler_[scheduler_name[1]].step(sum(losses[-batch_size:]))
                                else:
                                    scheduler_[scheduler_name[1]].step()
                            else:
                                raise NotImplementedError("No implementation for more than 2 "
                                                          "schedulers")

                        list_scheduler.append(optimizer.param_groups[0]["lr"])
                # ----------------------------------------------------------------------------------
                ## Save checkpoint at every epoch
                # ----------------------------------------------------------------------------------
                if save_checkpoint and ((i + 1) % batch_size == 0):
                    checkpoint = {
                        "nb_epochs_finished": (i + 1) // batch_size,
                        "log_alpha": self.log_alpha,
                        "optimizer_state": optimizer.state_dict(),
                        "scheduler_state": (
                            [scheduler_[elem].state_dict() for elem in scheduler_name]
                            if scheduler_name is not None else None
                        ),
                        "losses": losses,
                        "list_scheduler": list_scheduler,
                        "all_indices": [target_idx_np, list_idx],
                        "list_perm_matrices": mats,
                        "list_log_alphas": list_log_alpha
                    }
                    torch.save(checkpoint, folder_path.parents[0] / "checkpoint.pth")
                # ----------------------------------------------------------------------------------
                ## Stopping rule
                # ----------------------------------------------------------------------------------
                # If the permutation matrix is the same for (epochs/4) consecutive epochs, stops the
                # training. It works only if there is noise, in all the non-noisy cases there are
                # long periods in which the permutation matrix doesn't change (constant loss). Also
                # first 50% of epochs is always trained, early stopping is possible only after that.
                if early_stopping and noise and ((i + 1) % batch_size == 0) and ((i + 1) // batch_size >= epochs // 2):
                    is_same_perm = [np.all(list_idx[-1] == list_idx[-batch_size * (i + 1)])
                                    for i in range(epochs // 4)]
                    if np.all(is_same_perm):
                        print(f"Early stopping at epoch: {(i + 1) // batch_size}")
                        break

        # ------------------------------------------------------------------------------------------
        ## Show timing histograms
        # ------------------------------------------------------------------------------------------
        if time_processes:
            fig, axes = plt.subplots(figsize=(10, 5),
                                     ncols=3,
                                     constrained_layout=True)
            axes[0].hist(delta1, bins=20)
            axes[0].set_title("Gumbel-Sinkhorn")
            axes[1].hist(delta2, bins=20)
            axes[1].set_title("Forward")
            axes[2].hist(delta3, bins=20)
            axes[2].set_title("Backward")
            fig.savefig(folder_path / "time_processes.svg")
            plt.show()

        return losses, list_scheduler, [target_idx_np, list_idx], [mats, mats_gs], list_log_alpha

    def target_loss(self, input_left, input_right, fixed_pairings=None,
                    positive_examples=None, batch_size=1, offset_pos_emb_from=None,
                    offset_pos_emb_by=0, kind_loss="average", **kwargs):
        """
        Function that computes the target value of the loss function using ordered pairs of input_left
        and input_right.
        `kind_loss` defines the loss function:
            = "average" or None -> average loss over `batch_size` iterations
            = "per_sequence" -> per-sequence average loss over `batch_size` iterations
            = "perplexity" -> perplexity of the non-masked input concatenation.
        """
        if kind_loss != "perplexity":
            self._validator(input_left, input_right, fixed_pairings=fixed_pairings)
            pbar = tqdm(range(batch_size))
            pbar.set_description("Computing target loss")
        else:
            pbar = range(batch_size)
        # Input MSAs
        input_left = input_left.to(self.device)
        input_right = input_right.to(self.device)

        self._validate_p_mask()

        with torch.set_grad_enabled(False):
            target_loss_val = []
            torch.cuda.empty_cache()
            for i in pbar:
                only_masked_part = True
                if kind_loss == "perplexity":
                    orig_p_mask = self.p_mask
                    self.p_mask = 0
                    only_masked_part = False
                # logits, mask = self(
                #     input_left, input_right,
                #     positive_examples=positive_examples, offset_pos_emb_from=offset_pos_emb_from,
                #     offset_pos_emb_by=offset_pos_emb_by, only_masked_part=only_masked_part
                # )
                if kind_loss == "per_sequence":
                    loss_ = []
                    for ind in range(logits.shape[1]):
                        new_mask = mask[:, [ind], ...]
                        ll = self.Loss(logits[:, [ind], ...][new_mask].reshape((-1, logits.shape[-1])),
                                       input_left[:, [ind], 1:, :][new_mask].reshape((-1, input_left.shape[-1])))
                        loss_.append(ll.item())
                    loss_ = np.array(loss_)
                    target_loss_val += [loss_]
                elif kind_loss == "perplexity":
                    loss = self.Loss_flat(logits.reshape((-1, logits.shape[-1])),
                                          torch.cat((input_left[:, :, 1:, :],
                                                     input_right[:, :, 1:, :]), dim=2).reshape((-1, input_left.shape[-1])))
                    loss = DCN(loss)
                    loss = loss.reshape((input_left.shape[1], -1))
                    loss_ = loss.mean(1)
                    loss_ = np.exp(loss_)
                    target_loss_val += [loss_]
                    # reset masking probability to original value
                    self.p_mask = orig_p_mask
                    # only one iteration is needed
                    break
                else:
                    input_for_CE_loss = input_left if self.only_mask_left else torch.cat((input_left, input_right[..., 1:, :]), dim=2)
                    for p_mask in self._p_mask:
                        logits, mask = self(
                            input_left, input_right, positive_examples=positive_examples,
                            p_mask=p_mask, offset_pos_emb_from=offset_pos_emb_from,
                            offset_pos_emb_by=offset_pos_emb_by, only_masked_part=only_masked_part
                        )
                        loss_ = self.Loss(logits[mask], input_for_CE_loss[..., 1:, :][mask]).item()
                        target_loss_val += [loss_]

        return target_loss_val

    # @profile
    def rw_train(self,
              input_left,
              input_right,
              fixed_pairings=None,  # [[(i, j), ...], ...]
              positive_examples=None,
              std_init=0.1,
              epochs=1,
              tau=1.,
              n_sink_iter=20,
              batch_size=1,
              use_rand_perm=False,
              keep_working_hard_perm=False,
              offset_pos_emb_from=None,
              offset_pos_emb_by=0,
              tar_loss=None,
              folder_path=None,
              showfig=False,
              only_loss_plot=False,
              **kwargs):
        self._validator(input_left, input_right, fixed_pairings=fixed_pairings)
        if not sum(self._effective_depth_not_fixed):
            print("No parameters available to optimize, pairings are fixed by the input.")
            return None
        self.std_init = std_init

        self._validate_p_mask()

        base_params = {"noise": False, "noise_mat": None}
        sinkhorn_params = {"tau": tau, "n_iter": n_sink_iter}

        # Initialize log_alpha given fixed pairings
        self._init_log_alpha()
        # ------------------------------------------------------------------------------------------
        ## Useful functions
        # ------------------------------------------------------------------------------------------
        def _identity(x, **kwargs):
            return x

        def _noise_mat():
            if self._is_depth_ndarray:
                return [None for la in self.log_alpha]
            return None
        
        def _impl_func(func):
            # Apply `func` either to the whole log_alpha or just to the blocks for permutation over
            # species
            def _case_restricted_to_species(log_alpha, **params):
                # Block matrix for permutations within species
                noise_mat = params.pop("noise_mat")  # List of noise matrices
                rand_perm = params.pop("rand_perm")  # List of random permutations
                return torch.block_diag(
                    *[func(la, noise_mat=nm, rand_perm=rp, **params) if la.size(0) else la
                      for la, nm, rp in zip(log_alpha, noise_mat, rand_perm)]
                )

            def _case_unrestricted(log_alpha, **params):
                # Else, permutation matrix over all sequences
                # TODO INCLUDE NOISE AND RANDOM PERMUTATIONS
                return func(log_alpha, **params) if log_alpha.size(0) else log_alpha

            if self._is_depth_ndarray:
                return _case_restricted_to_species
            return _case_unrestricted

        def _keep_working_hard_perm_func(gs_matching_mat, prev_gs_matching_mat=None):
            # Keep same permutation if the frobenius norm of the previous one is bigger or
            # equal than the new one (without Noise)
            if prev_gs_matching_mat is None:
                pass
            elif self._is_depth_ndarray:
                d_start = 0
                for d, la in zip(self.depth, self.log_alpha):
                    d_end = d_start + d
                    gs_matching_submat = gs_matching_mat[d_start:d_end, d_start:d_end]
                    prev_gs_matching_submat = prev_gs_matching_mat[d_start:d_end, d_start:d_end]
                    frob = gs_matching_submat.T.matmul(la).diagonal().sum()
                    prev_frob = prev_gs_matching_submat.T.matmul(la).diagonal().sum()
                    if prev_frob >= frob:
                        # Frobenius norm should be maximised, use old matching if not worse
                        gs_matching_mat[d_start:d_end, d_start:d_end] = prev_gs_matching_submat
                    d_start = d_end
            else:
                frob = gs_matching_mat.T.matmul(self.log_alpha).diagonal().sum()
                prev_frob = prev_gs_matching_mat.T.matmul(self.log_alpha).diagonal().sum()
                if prev_frob >= frob:
                    # Frobenius norm should be maximised, use old matching if not worse
                    gs_matching_mat[:, :] = prev_gs_matching_mat

            return gs_matching_mat

        def _rand_perm():
            if self._is_depth_ndarray:
                if use_rand_perm:
                    rand_perm = []
                    for la in self.log_alpha:
                        n = la.shape[0]
                        rp = []
                        for _ in range(2):
                            rp_i = torch.zeros_like(la, device=self.device)
                            rp_i[torch.arange(n), torch.randperm(n)] = 1
                            rp.append(rp_i)
                        rand_perm.append(rp)
                else:
                    rand_perm = [None] * len(self.log_alpha)
            else:
                if use_rand_perm:
                    rand_perm = []
                    n = self.log_alpha.shape[0]
                    for _ in range(2):
                        rand_perm_i = torch.zeros_like(self.log_alpha, device=self.device)
                        rand_perm_i[torch.arange(n), torch.randperm(n)] = 1
                        rand_perm.append(rand_perm_i)
                else:
                    rand_perm = None
            return rand_perm

        if keep_working_hard_perm:
            processing_ = _keep_working_hard_perm_func
        else:
            processing_ = _identity

        gumbel_matching_ = _impl_func(gumbel_matching)
        gumbel_sinkhorn_ = _impl_func(gumbel_sinkhorn)

        # ------------------------------------------------------------------------------------------
        ## Input MSAs and initial variables
        # ------------------------------------------------------------------------------------------
        input_left = input_left.to(self.device).requires_grad_(False)
        input_right = input_right.to(self.device).requires_grad_(False)

        # Lists of parameters
        losses = []
        mats, mats_gs = [], []
        list_idx = []
        list_log_alpha = []
        gs_matching_mat = None
        target_idx = torch.arange(self._depth_total, dtype=torch.float, device=self.device)
        target_idx_np = DCN(target_idx)

        # ------------------------------------------------------------------------------------------
        ## Start training
        # ------------------------------------------------------------------------------------------
        iterations = epochs * batch_size
        with torch.no_grad():
            for i in tqdm(range(iterations + batch_size)):
                # ----------------------------------------------------------------------------------
                ## Block Matrices for permutations in species
                # ----------------------------------------------------------------------------------
                if i % batch_size == 0:
                    # Save log_alpha
                    _log_alpha = torch.full((self._depth_total, self._depth_total), torch.nan,
                                            dtype=torch.float, device=self.device)
                    _log_alpha.masked_scatter_(
                        self._effective_mask_not_fixed,
                        torch.block_diag(*self.log_alpha)
                        if self._is_depth_ndarray else self.log_alpha
                    )
                    list_log_alpha.append(DCN(_log_alpha))
                    # Random shufflings only every `batch_size` iterations
                    rand_perm = _rand_perm()
                    noise_mat = _noise_mat()
                # Set value of noise correction
                new_noise_factor = 0
                # ----------------------------------------------------------------------------------
                ## Compute permutation matrices
                # ----------------------------------------------------------------------------------
                params = {**base_params,
                          **{"noise_mat": noise_mat, "noise_factor": new_noise_factor,
                             "rand_perm": rand_perm}}
                gs_matching_mat_not_fixed = processing_(gumbel_matching_(self.log_alpha, **params),
                                                        prev_gs_matching_mat=gs_matching_mat)
                params.update(sinkhorn_params)
                gs_mat_not_fixed = gumbel_sinkhorn_(self.log_alpha, **params)
                if fixed_pairings is not None:
                    gs_matching_mat = torch.zeros(self._depth_total, self._depth_total,
                                                  dtype=torch.float, device=self.device)
                    _gs_mat = torch.zeros_like(gs_matching_mat, device=self.device, requires_grad=True)
                    gs_mat = _gs_mat.clone()
                    start = 0
                    for (species_size,
                         species_fixed_pairings) in zip(self._depth,
                                                        self._effective_fixed_pairings_zip):
                        gs_matching_mat[start:, start:][species_fixed_pairings] = 1.
                        gs_mat[start:, start:][species_fixed_pairings] = 1.
                        start += species_size
                    gs_mat.masked_scatter_(self._effective_mask_not_fixed, gs_mat_not_fixed)
                    gs_matching_mat.masked_scatter_(self._effective_mask_not_fixed,
                                                    gs_matching_mat_not_fixed)
                else:
                    gs_matching_mat = gs_matching_mat_not_fixed
                    gs_mat = gs_mat_not_fixed
                # Save hard or soft permutation matrix
                if i % batch_size == 0:
                    mats.append(DCN(gs_matching_mat))
                    mats_gs.append(DCN(gs_mat))
                # Save permuted indexes
                list_idx.append(DCN(torch.einsum("pq,p->q", (gs_matching_mat, target_idx))))
                # ----------------------------------------------------------------------------------
                ## Permute sequences of input_left.
                # ----------------------------------------------------------------------------------
                # input_left_ord = MSA_inverse_permutation(input_left, gs_mat)
                input_left_ord = MSA_inverse_permutation(input_left, gs_matching_mat)
                # ----------------------------------------------------------------------------------
                ## Get output logits of MSA Transformer for the permuted, masked input
                # ----------------------------------------------------------------------------------
                loss = torch.tensor(0., device=self.device)
                input_for_CE = input_left_ord if self.only_mask_left else torch.cat((input_left_ord, input_right[..., 1:, :]), dim=2)
                for p_mask in self._p_mask:
                    logits, mask = self(
                        input_left_ord, input_right, positive_examples=positive_examples,
                        p_mask=p_mask, offset_pos_emb_from=offset_pos_emb_from,
                        offset_pos_emb_by=offset_pos_emb_by)
                    # MLM loss
                    loss = loss + self.Loss(logits[mask], input_for_CE[..., 1:, :][mask])
                loss = loss / batch_size
                pure_loss = loss.item()
                # Save loss values
                losses.append(pure_loss * batch_size)
                # Update log_alpha
                if i < iterations:
                    if ((i + 1) % batch_size == 0) or ((i + 1) == iterations):                
                        self._init_log_alpha()
                        self.log_alpha = [elem.requires_grad_(False) for elem in self.log_alpha]

                #      plot and save at every batch_size     or       no plots and save at last iteration
                if (((i + 1) % batch_size == 0) and showfig) or ((i == iterations + batch_size - 1) and not showfig):
                    self.plot_real_time(
                        i, DCN(gs_matching_mat), DCN(gs_mat), list_idx, target_idx_np,
                        list_log_alpha, losses, batch_size, epochs,
                        0, tar_loss, new_noise_factor, folder_path,
                        showfig, only_loss_plot)

        return losses, None, [target_idx_np, list_idx], [mats, mats_gs], list_log_alpha

In [ ]:
show_doc(Permutations_MLM)

In [ ]:
#| hide
# !nbdev_export
# !nbdev_test
# !nbdev_clean
# !nbdev_prepare